# Day 4 - Lab 1: Automated Testing & Quality Assurance (Solution)

**Objective:** Generate a comprehensive `pytest` test suite for the database-connected FastAPI application, including tests for happy paths, edge cases, and tests that use advanced fixtures for database isolation.

**Introduction:**
This solution notebook provides the complete prompts and explanations for generating a robust test suite. It covers generating simple tests, brainstorming edge cases, and creating the necessary fixtures for professional-grade database testing.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

**Explanation:**
We load the application's source code to provide the LLM with the necessary context to write accurate tests. A good prompt for test generation should always include the code that needs to be tested.

In [1]:
import sys
import os

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output, prompt_enhancer

# Initialize separate LLM clients for each lab task so we can experiment with different models per task.
# You can change the model_name for each client to a model available in your environment or the RECOMMENDED_MODELS table.
# Use a stronger model for generation of structured tests, and lighter models for edge cases/fixtures to save quota in examples.
happy_client, happy_model, happy_provider = setup_llm_client(model_name="gpt-4o")
edge_client, edge_model, edge_provider = setup_llm_client(model_name="gpt-4o-mini")
fixture_client, fixture_model, fixture_provider = setup_llm_client(model_name="gpt-4o-mini")

print(f"Happy-path model: {happy_model} (provider: {happy_provider})")
print(f"Edge-case model: {edge_model} (provider: {edge_provider})")
print(f"Fixture model: {fixture_model} (provider: {fixture_provider})")

# Load the application code from Day 3 to provide context for test generation
app_code = load_artifact("app/main.py")
if not app_code:
    print("Warning: Could not load app/main.py. Lab may not function correctly.")


✅ LLM Client configured: Using 'openai' with model 'gpt-4o'
✅ LLM Client configured: Using 'openai' with model 'gpt-4o-mini'
✅ LLM Client configured: Using 'openai' with model 'gpt-4o-mini'
Happy-path model: gpt-4o (provider: openai)
Edge-case model: gpt-4o-mini (provider: openai)
Fixture model: gpt-4o-mini (provider: openai)


## Step 2: The Challenges - Solutions

### Challenge 1 (Foundational): Generating "Happy Path" Tests

**Explanation:**
This prompt asks for the most straightforward type of test: one that verifies the application works as expected when given valid input. We specifically ask for tests for the `POST` and `GET` endpoints. The prompt includes the full application code as context, which is crucial for the LLM to understand the API's structure, expected payloads, and responses.

In [2]:
happy_path_tests_prompt = f"""
You are a Senior QA Engineer writing tests for a FastAPI application using pytest.

Based on the application code provided below, please generate two 'happy path' test functions in a single Python script:
1. A test named `test_create_user` for the `POST /users/` endpoint. It should create a user and assert that the status code is 200 and the response email matches the input.
2. A test named `test_read_users` for the `GET /users/` endpoint. It should first create a user and then assert the status code is 200 and that the response is a list containing at least one user.

**Application Code Context:**
```python
{app_code}
```

Your response should be only the raw Python code for the tests, including necessary imports like `TestClient` from `fastapi.testclient`.
"""

print("--- Generating Happy Path Tests ---")
if app_code:
    # Enhance the prompt for more consistent and focused output
    enhanced_happy_prompt = prompt_enhancer(happy_path_tests_prompt, model_name=happy_model, client=happy_client, api_provider=happy_provider)
    generated_happy_path_tests = get_completion(enhanced_happy_prompt, happy_client, happy_model, happy_provider)
    cleaned_tests = clean_llm_output(generated_happy_path_tests, language='python')
    print(cleaned_tests)
    save_artifact(cleaned_tests, "tests/test_main_simple.py")
else:
    print("Skipping test generation because app code is missing.")


--- Generating Happy Path Tests ---
from fastapi.testclient import TestClient
from .main import app  # Assuming the FastAPI app is in a file named main.py

client = TestClient(app)

def test_create_user():
    # Send a request to create a user
    response = client.post("/users/", json={"email": "test@example.com", "password": "testpassword"})
    # Assert that the response status code is 200
    assert response.status_code == 200
    # Assert that the response email matches the input email
    assert response.json()["email"] == "test@example.com"

def test_read_users():
    # Create a user first
    client.post("/users/", json={"email": "test@example.com", "password": "testpassword"})
    
    # Send a request to retrieve the list of users
    response = client.get("/users/")
    # Assert that the response status code is 200
    assert response.status_code == 200
    # Assert that the response contains a list with at least one user
    assert len(response.json()) > 0
✅ Successfully sa

### Challenge 2 (Intermediate): Generating Edge Case Tests

**Explanation:**
Good testing goes beyond the happy path. This prompt asks the LLM to think about what could go wrong. We specifically request tests for two common failure modes: creating a duplicate resource (which should be disallowed) and requesting a resource that doesn't exist. This demonstrates how AI can be used as a creative partner to brainstorm potential failure points.

In [3]:
edge_case_tests_prompt = f"""
You are a QA Engineer focused on identifying edge cases.

Based on the FastAPI application code provided, write two test functions for common error scenarios:
1.  A test named `test_create_user_duplicate_email` that attempts to create a user with an email that already exists. It must assert that the API returns a 400 status code.
2.  A test named `test_read_user_not_found` that attempts to GET a user with an ID that does not exist (e.g., 999). It must assert that the API returns a 404 status code.

**Application Code Context:**
```python
{app_code}
```

Output only the raw Python code for these two test functions.
"""

print("--- Generating Edge Case Tests ---")
if app_code:
    enhanced_edge_prompt = prompt_enhancer(edge_case_tests_prompt, model_name=edge_model, client=edge_client, api_provider=edge_provider)
    generated_edge_case_tests = get_completion(enhanced_edge_prompt, edge_client, edge_model, edge_provider)
    cleaned_edge_case_tests = clean_llm_output(generated_edge_case_tests, language='python')
    print(cleaned_edge_case_tests)
    # In a real scenario, you'd append these to your test file.
else:
    print("Skipping test generation because app code is missing.")


--- Generating Edge Case Tests ---
The provided code consists of two test functions designed to validate the behavior of a user management API. Let's break down each test case:

1. **Test for Duplicate Email Registration**:
   - The first function, `test_create_user_duplicate_email`, tests the scenario where a user tries to register with an email that is already in use.
   - It begins by creating a user with a unique email (`"test@example.com"`).
   - It then attempts to create another user with the same email, expecting the API to respond with a status code of 400 Bad Request and a specific error message indicating that the email has already been registered.

2. **Test for Non-Existent User Retrieval**:
   - The second function, `test_read_user_not_found`, tests the scenario where a user attempts to retrieve a user by an ID that does not exist in the system (in this case, ID `999`).
   - The test expects the API to respond with a status code of 404 Not Found and a message indicating t

### Challenge 3 (Advanced): Testing with an Isolated Database Fixture

**Explanation:**
This is the most advanced and most important concept in this lab. A `pytest` fixture is a function that runs before each test to set up a specific state or resource. 

The prompt asks the LLM to generate a fixture that creates an isolated, in-memory database for testing. This is a best practice because it ensures tests are independent and don't interfere with each other or with the real development database. 

We specifically instruct the LLM to save this fixture in `tests/conftest.py`. `conftest.py` is a special file that pytest automatically discovers. Fixtures defined here are globally available to all test files in the same directory and subdirectories, making it the ideal place to put shared setup code like a database connection. 

Finally, we ask the LLM to refactor our original happy-path tests to *use* this fixture by simply adding it as a function argument.

In [4]:
db_fixture_prompt = f"""
You are an expert in Python testing with pytest and FastAPI.

I need to create a `pytest` fixture to provide an isolated, in-memory SQLite database session for each test run. This is a critical best practice for testing database-connected applications.

Please generate the Python code for a file named `tests/conftest.py` that contains this fixture.

The fixture should:
1. Be named `db_session`.
2. Configure a SQLAlchemy engine for an in-memory SQLite database.
3. Create all database tables before the tests run.
4. Yield a database session.
5. Clean up the database tables after the tests are complete.

**Application Code Context:**
```python
{app_code}
```

Output only the raw Python code for the `conftest.py` file.
"""

print("--- Generating Pytest DB Fixture for conftest.py ---")
if app_code:
    enhanced_fixture_prompt = prompt_enhancer(db_fixture_prompt, model_name=fixture_model, client=fixture_client, api_provider=fixture_provider)
    generated_db_fixture = get_completion(enhanced_fixture_prompt, fixture_client, fixture_model, fixture_provider)
    cleaned_fixture = clean_llm_output(generated_db_fixture, language='python')
    print(cleaned_fixture)
    save_artifact(cleaned_fixture, "tests/conftest.py")
else:
    print("Skipping fixture generation because app context is missing.")

refactor_tests_prompt = f"""
You are a QA Engineer refactoring a test suite to use a new database fixture.

Given the following tests and the knowledge that a fixture named `db_session` and a `TestClient` instance named `client` are now available from `conftest.py`, please rewrite the tests to use them. The tests should no longer create their own client instances.

**Original Tests:**
```python
{generated_happy_path_tests if 'generated_happy_path_tests' in locals() else ''}
```

**Application Code Context:**
```python
{app_code}
```

Output only the raw Python code for the refactored tests.
"""

print("\n--- Generating Refactored Tests for test_main_with_fixture.py ---")
if app_code and 'generated_happy_path_tests' in locals():
    enhanced_refactor_prompt = prompt_enhancer(refactor_tests_prompt, model_name=fixture_model, client=fixture_client, api_provider=fixture_provider)
    refactored_tests = get_completion(enhanced_refactor_prompt, fixture_client, fixture_model, fixture_provider)
    cleaned_refactored_tests = clean_llm_output(refactored_tests, language='python')
    print(cleaned_refactored_tests)
    save_artifact(cleaned_refactored_tests, "tests/test_main_with_fixture.py")
else:
    print("Skipping test refactoring because app context or original tests are missing.")


--- Generating Pytest DB Fixture for conftest.py ---
import pytest
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, scoped_session
from myapp.database import Base  # Adjust the import according to your project structure

@pytest.fixture(scope='function')
def db_session():
    engine = create_engine('sqlite:///:memory:')
    Base.metadata.create_all(engine)
    
    Session = scoped_session(sessionmaker(bind=engine))
    session = Session()

    yield session

    session.close()
    Session.remove()
    Base.metadata.drop_all(engine)
✅ Successfully saved artifact to: artifacts/tests/conftest.py

--- Generating Refactored Tests for test_main_with_fixture.py ---
import pytest
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, scoped_session
from myapp.database import Base  # Adjust the import according to your project structure

@pytest.fixture(scope='function')
def db_session():
    engine = create_engine('sqlite:///:memory:')
    

## Lab Conclusion

Fantastic work! You have built a comprehensive test suite for your API, moving from simple happy path tests to advanced, isolated database testing. You've learned how to use AI to brainstorm edge cases and generate complex fixtures. Having a strong test suite like this gives you the confidence to make changes to your application without fear of breaking existing functionality.

> **Key Takeaway:** Using AI to generate tests is a massive force multiplier for quality assurance. It excels at creating boilerplate test code, brainstorming edge cases, and generating complex setup fixtures, allowing developers to build more reliable software faster.